In [2]:
# pip install accelerate bitsandbytes
import torch
import requests
from PIL import Image
from transformers import Blip2Processor, Blip2Model

model_name="Salesforce/blip2-flan-t5-xl"
# model_name="Salesforce/blip2-opt-6.7b-coco"
processor = Blip2Processor.from_pretrained(model_name)
model = Blip2Model.from_pretrained(model_name, load_in_8bit=True, device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
from dotenv import load_dotenv
import os
from tqdm import tqdm
import json

load_dotenv()  # This loads the environment variables from .env

api_key = os.getenv('HGF_KEY')

from datasets import load_dataset
winoground = load_dataset('facebook/winoground', token=api_key)['test']

In [4]:
example = winoground[7]
# winoground dataset images are already in RGB mode
raw_image_0 = example['image_0'] 

In [5]:
inputs = processor(images=raw_image_0, return_tensors="pt")
qformer_outputs = model.get_qformer_features(**inputs)

In [7]:
qformer_outputs.last_hidden_state.shape

torch.Size([1, 32, 768])